## Code to get most influential references or citations
- Documentation of API: https://api.semanticscholar.org/api-docs/#tag/Author-Data/operation/post_graph_get_authors

In [ ]:
import requests
import time

/home/feynman/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
paper_id = "b9b220b485d2add79118ffdc2aaa148b67fa53ef" 


url_string = f"https://api.semanticscholar.org/graph/v1/paper/{paper_id}/citations?fields=title,authors,isInfluential"


# function to load the json file that is on the url
def load_json_from_url(url):
    response = requests.get(url)
    data = response.json()
    return data

data = load_json_from_url(url_string)

In [ ]:

def get_paper_refs_or_cits(paper_id, fields="title,authors,isInfluential", limit=100, use_references=True):
    """
    Fetch all references for a paper, handling pagination.
    
    Args:
        paper_id: The Semantic Scholar paper ID
        fields: Comma-separated fields to include in the response
        limit: Number of items to fetch per request
        
    Returns:
        Tuple of (all_references, influential_references)
    """

    suffix = "references" if use_references else "citations"
    base_url = f"https://api.semanticscholar.org/graph/v1/paper/{paper_id}/{suffix}"
    url = f"{base_url}?fields={fields}&limit={limit}"
    
    all_references = []
    offset = 0
    
    while True:
        # Add offset to URL if not first request
        paginated_url = f"{url}&offset={offset}" if offset > 0 else url
        
        try:
            response = requests.get(paginated_url)
            response.raise_for_status()  # Raise exception for HTTP errors
            data = response.json()
            
            # Add fetched references to our collection
            references = data.get('data', [])
            all_references.extend(references)
            
            # Check if there are more references to fetch
            if 'next' in data:
                offset = data['next']
                # Add a small delay to avoid hitting rate limits
                time.sleep(0.5)
            else:
                # No more pages, we're done
                break
                
        except requests.exceptions.RequestException as e:
            print(f"Error fetching references: {e}")
            break
    
    # Filter for influential references
    influential_references = [ref for ref in all_references if ref.get('isInfluential', False)]
    
    return all_references, influential_references

# Example usage
# if __name__ == "__main__":
use_references = False

paper_id = "b9b220b485d2add79118ffdc2aaa148b67fa53ef"
all_refs, influential_refs = get_paper_refs_or_cits(
    paper_id,
    fields="title,authors,isInfluential,url",
    use_references=use_references
)

print(f"Total references: {len(all_refs)}")
print(f"Influential references: {len(influential_refs)}")

Total references: 234
Influential references: 13


In [20]:
use_references = False

# Print first few influential references
for i, ref in enumerate(influential_refs[:5]):
    print(f"\nInfluential Reference {i+1}:")
    if use_references is True:
        paper = ref.get('citedPaper', {})
    else:
        paper = ref.get('citingPaper', {})
        
    print(f"Title: {paper.get('title')}")
    print(f"Authors: {', '.join(a.get('name', '') for a in paper.get('authors', []))}")
    print(f"URL: {paper.get('url')}")


Influential Reference 1:
Title: SAC-GLAM: Improving Online RL for LLM agents with Soft Actor-Critic and Hindsight Relabeling
Authors: Loris Gaven, Clément Romac, Thomas Carta, Sylvain Lamprier, Olivier Sigaud, Pierre-Yves Oudeyer
URL: https://www.semanticscholar.org/paper/10efca368b696136714fda841d0848f2c231516d

Influential Reference 2:
Title: Survey on Large Language Model-Enhanced Reinforcement Learning: Concept, Taxonomy, and Methods
Authors: Yuji Cao, Huan Zhao, Yuheng Cheng, Ting Shu, Guolong Liu, Gaoqi Liang, Junhua Zhao, Yun Li
URL: https://www.semanticscholar.org/paper/c44471e846846bde281779405a3b5c132fd60b00

Influential Reference 3:
Title: MLDT: Multi-Level Decomposition for Complex Long-Horizon Robotic Task Planning with Open-Source Large Language Model
Authors: Yike Wu, Jiatao Zhang, Nan Hu, LanLing Tang, Guilin Qi, Jun Shao, Jie Ren, Wei Song
URL: https://www.semanticscholar.org/paper/9ad90fb50754c11e60c208f1f698eed617d75d1f

Influential Reference 4:
Title: Unleashing th